# The Gladius Data Model

### Data Model Structure


Collection: "clubs"

Document: "club_id"
Club data fields
Sub-collection: "events"
Document: "event_id"
Event data fields
Sub-collection: "occurrences"
Document: "occurrence_id"
Occurrence data fields
Sub-collection: "attendance"
Document: "attendance_id"
Field: "user_id" (to reference the attending user)
Field: "score" (record the score for the user at this occurrence)

Collection: "users"

Document: "user_id"
User data fields
Sub-collection: "scores"
Document: "score_id"
Field: "club_id" (to reference the club)
Field: "event_id" (to reference the event)
Field: "occurrence_id" (to reference the occurrence)
Field: "score" (the user's score for this event occurrence)


In [80]:
# Installs
#!pip install firebase-admin

In [81]:

# Import firebase_admin and initialize the app
import firebase_admin
from firebase_admin import auth, credentials, firestore
import random


Authentication

In [82]:
if not firebase_admin._apps:
    cred = credentials.Certificate("../firebase-adminsdk.json")
    default_app = firebase_admin.initialize_app(cred)
# Get a reference to the firestore database
db = firestore.client()

# Create users with authentication

### Delete all auth users

In [83]:
user_list = auth.list_users()

# Iterate over each user and delete them
for user in user_list.users:
    auth.delete_user(user.uid)
    print(f"Deleted user: {user.uid}")

print("All users have been deleted.")

Deleted user: JC7wqDBCm1areGebpEAP9H4ZFCj1
Deleted user: JsgwlIfnJ6c6QqavCQEv3XiADXC2
Deleted user: KwDzxrTIqJVqFJfwQp5DS2K9rED2
Deleted user: O3VaXyqvVGP7Lvw4nKWOomofMg83
Deleted user: OuqfrySfQmSeWYR0qfd77oSPSDS2
Deleted user: VEK5BdcbMahpB6MXnq8eQ6RxkW33
Deleted user: VQRdv8TU5mer9ektfPC6GdC7Jfp2
Deleted user: gQMT2KpqPJX6AN6jRvurUOntyl63
Deleted user: lIIQh7LcfRS1fttNMzwgP2gvIKT2
Deleted user: rrxFS9SSFCbDoqjZh7Ixb18VR7j1
Deleted user: yNy3xmcp19RaJhekiNYFFAi3udy2
All users have been deleted.


### Create new auth users

In [84]:
# Create a list of users to add
users = ["Mare", "Jüri", "Liisa", "Mart", "Evelin", "Tõnis", "Mari", "Henrik", "Triin", "Jaak", 'Bob']


user_uuids = {}
# Loop through the users and create them with a default password
for user in users:
  try:
    # Create the user with the email and password
    user_record = auth.create_user(
      email=f'{user.lower()}@example.com',
      password='123456'
    )
    user_uuids[user] = user_record.uid
    # Print the user ID
    print(f'Successfully created new user: {user_record.uid}')
  except firebase_admin.auth.AuthError as e:
    # Handle any errors
    print(f'Error creating new user: {e}')

Successfully created new user: OYsTIWxITJTDRrMRX3FOZ4zeX3F2
Successfully created new user: f0bOQGOlLAQp4gzjJexbSaKoRp13
Successfully created new user: oq7zFTYUdqczFw7tsM5V4SXYYkU2
Successfully created new user: uug2mnETM0PrZntIrf6nkw2JAPb2
Successfully created new user: ZmJ8Mb0bEoZUqOfm8SH8XKjsNrP2
Successfully created new user: k4s5ZsOWF6SV8h3Gh4aSnCBelWj2
Successfully created new user: mxFfl1jym8NWkcxjxybuXzsK5Xp1
Successfully created new user: 6uuGNDujf3faRDxPldlMSl9Wnns2
Successfully created new user: rsuqCcPclYP7RNApjFSonxzhGal1
Successfully created new user: GYHcWELQkFMpd9NidY8fyv3zER43
Successfully created new user: WA5Gk2q1sZduXH3BNDNWVHJw8Ei2


### Delete user collection

In [85]:
## Delete user collection
# Function to delete a collection and its documents
def delete_collection(collection_ref, batch_size):
    docs = collection_ref.limit(batch_size).stream()
    deleted = 0

    for doc in docs:
        doc.reference.delete()
        deleted += 1

    if deleted >= batch_size:
        return delete_collection(collection_ref, batch_size)

# Specify the "users" collection to be deleted
users_collection_ref = db.collection("users")

# Delete the "users" collection and its documents
delete_collection(users_collection_ref, 500)

print("Users collection deleted successfully.")

Users collection deleted successfully.


## Add users collection

In [86]:
# Create a collection for users
users_collection = db.collection("users")

user_clubs_roles = {}

for user in users:
    user_data = {
        "name": user,
        "email": f"{user.lower()}@example.com",
        "age": random.randint(18, 60),
        "occupation": random.choice(["Student", "Engineer", "Teacher", "Artist"]),
        "is_active": random.choice([True, False]),
        "clubs_roles": [],  # initialize an empty list for clubs and roles
    }

    user_ref = users_collection.document(user_uuids[user])
    user_ref.set(user_data)
    print(f"Successfully created user: {user_uuids[user]}")

    if user in ["Bob", "Mare"]:
        user_clubs_roles[user_uuids[user]] = [{"club_id": "1", "role": "owner"}]
    else:
        user_clubs_roles[user_uuids[user]] = [{"club_id": "1", "role": "athlete"}]

# After creating all users, update the users with their clubs and roles
for user_uuid, clubs_roles in user_clubs_roles.items():
    user_ref = users_collection.document(user_uuid)
    user_ref.update({"clubs_roles": clubs_roles})


Successfully created user: OYsTIWxITJTDRrMRX3FOZ4zeX3F2
Successfully created user: f0bOQGOlLAQp4gzjJexbSaKoRp13
Successfully created user: oq7zFTYUdqczFw7tsM5V4SXYYkU2
Successfully created user: uug2mnETM0PrZntIrf6nkw2JAPb2
Successfully created user: ZmJ8Mb0bEoZUqOfm8SH8XKjsNrP2
Successfully created user: k4s5ZsOWF6SV8h3Gh4aSnCBelWj2
Successfully created user: mxFfl1jym8NWkcxjxybuXzsK5Xp1
Successfully created user: 6uuGNDujf3faRDxPldlMSl9Wnns2
Successfully created user: rsuqCcPclYP7RNApjFSonxzhGal1
Successfully created user: GYHcWELQkFMpd9NidY8fyv3zER43
Successfully created user: WA5Gk2q1sZduXH3BNDNWVHJw8Ei2


## Delete Club collection

In [97]:
def delete_collection(coll_ref, batch_size):
    docs = coll_ref.list_documents(page_size=batch_size)
    deleted = 0

    for doc in docs:
        print(f"Deleting doc {doc.id} => {doc.get().to_dict()}")
        
        # Delete subcollections of the document
        for subcoll in doc.collections():
            delete_collection(subcoll, batch_size)
            
        doc.delete()
        deleted = deleted + 1

    if deleted >= batch_size:
        return delete_collection(coll_ref, batch_size)

# Specify the "clubs" collection to be deleted
clubs_collection_ref = db.collection("clubs")

# Delete the "clubs" collection and its documents
delete_collection(clubs_collection_ref, 500)


Deleting doc 1 => {'description': 'A Learn to Earn Club', 'calendars': ['dcromp88@googlemail.com'], 'name': 'Gladius Club'}
Deleting doc Fpd69fT2GzaJ3yTrCrX8 => {'event_ids': [{'calendarId': 'dcromp88@googlemail.com', 'eventId': '4d06bu8godice3gdt7qs8k3cg4'}], 'member_uuids': ['WA5Gk2q1sZduXH3BNDNWVHJw8Ei2'], 'name': 'Bob'}
Deleting doc c1O7p8Ia3j4515xrKRAX => {'events': [{'calendarId': 'dcromp88@googlemail.com', 'eventId': '4d06bu8godice3gdt7qs8k3cg4'}], 'member_uuids': ['f0bOQGOlLAQp4gzjJexbSaKoRp13', 'uug2mnETM0PrZntIrf6nkw2JAPb2', 'ZmJ8Mb0bEoZUqOfm8SH8XKjsNrP2', 'rsuqCcPclYP7RNApjFSonxzhGal1', 'oq7zFTYUdqczFw7tsM5V4SXYYkU2', 'mxFfl1jym8NWkcxjxybuXzsK5Xp1', '6uuGNDujf3faRDxPldlMSl9Wnns2', 'OYsTIWxITJTDRrMRX3FOZ4zeX3F2', 'WA5Gk2q1sZduXH3BNDNWVHJw8Ei2', 'GYHcWELQkFMpd9NidY8fyv3zER43'], 'name': 'Random Group'}
Deleting doc vt68TAGCeMoQnXEFihYe => {'events': [{'calendarId': 'dcromp88@googlemail.com', 'eventId': '4d06bu8godice3gdt7qs8k3cg4'}], 'member_uuids': ['rsuqCcPclYP7RNApjFSonxzhGa

## Add club collection

In [98]:

# Create a collection for clubs
clubs_collection = db.collection("clubs")

# Create a document for a club with some data fields
club_id = '1'
club_data = {
    "name": "Gladius Club",
    "description": "A Learn to Earn Club",
    "calendars": ["dcromp88@googlemail.com"],
}
club_ref = clubs_collection.document(club_id)
club_ref.set(club_data)

# Create a users subcollection under the club
users_subcollection = club_ref.collection("members")

# Add users to the users subcollection
for user_uuid, clubs_roles in user_clubs_roles.items():
    for role in clubs_roles:
        if role["club_id"] == "1":
            # Get user data from the users collection
            user_doc_ref = users_collection.document(user_uuid)
            user_snapshot = user_doc_ref.get()
            user_data = user_snapshot.to_dict()

            # Create the user data to be stored in the subcollection
            user_subcollection_data = {
                "user": user_uuid,
                "name": user_data["name"],
                "email": user_data["email"],
                "role": role["role"],
            }
            users_subcollection.document(user_uuid).set(user_subcollection_data)



### Create groups

In [99]:

# The ID of the club under which the group will be created
club_id = "1"
club_ref = clubs_collection.document(club_id)

# Get all user UUIDs from the users collection
user_uuids = [doc.id for doc in users_collection.stream()]

# Select a random subset of users to be members of the group
member_uuids = random.sample(user_uuids, 10)  # choose 10 members randomly

group_data = {
    "name": "Random Group",  # the name of the group
    "member_uuids": member_uuids,  # the UUIDs of the group's members
    "event_ids": [  # list of events, where each event is an object with a calendarId and eventId
        {
            "calendarId": "dcromp88@googlemail.com",  # replace with actual calendarId
            "eventId": "4d06bu8godice3gdt7qs8k3cg4"  # replace with actual eventId
        },
        # more events can go here...
    ],
}

# Create the group in the groups subcollection under the club
groups_subcollection = club_ref.collection("groups")
group_ref = groups_subcollection.document()  # generate a random ID for the group
group_ref.set(group_data)

print(f"Successfully created group: {group_ref.id} under club: {club_id}")


Successfully created group: 2Dr32xte1JQ8CLD15H1Q under club: 1


In [90]:
# Query the group by its name
group_ref = clubs_collection.document(club_id).collection("groups").where("name", "==", "Random Group").get()

if len(group_ref) > 0:
    group_doc = group_ref[0]  # get the first matching group
    member_uuids = group_doc.to_dict()["member_uuids"]  # get the list of member UUIDs

    # Query the names of the members from the users collection
    member_names = [users_collection.document(uuid).get().to_dict()["name"] for uuid in member_uuids]

    print(f"Members of the group 'Random Group' are: {member_names}")
else:
    print("No group found with the name 'Random Group'")


/Users/davidcrompton/Documents/web-app-projects/.venv/lib/python3.9/site-packages/google/cloud/firestore_v1/base_collection.py:290: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  return query.where(field_path, op_string, value)


Members of the group 'Random Group' are: ['Evelin', 'Bob', 'Jaak', 'Triin', 'Liisa', 'Mare', 'Tõnis', 'Mari', 'Mart', 'Jüri']


## Add events to members

In [95]:
# Get the members subcollection under the club
members_subcollection = club_ref.collection("members")

# Specify calendarId and eventId
calendarId = "dcromp88@googlemail.com"
eventId = "4d06bu8godice3gdt7qs8k3cg4_20230715T110000Z"

# Add attendance records to the attendance subcollection of each member
for user_uuid in member_uuids:
    # Get the document reference for the member
    member_doc_ref = members_subcollection.document(user_uuid)
    
    # Create an attendance subcollection under the member
    attendance_subcollection = member_doc_ref.collection("attendance")

    # Prepare attendance data
    attendance_data = {
        "attended": True,
        "calendarId": calendarId,
        "coefficient": None,
        "eventId": eventId,
        "eventParentId": eventId.split('_')[0],  # splits eventId on '_' and gets the first part
        "score": None,
        "win": None,
    }

    # generate a document ID from "calendarId_eventId"
    document_id = f"{calendarId}_{eventId}"
    attendance_ref = attendance_subcollection.document(document_id)  
    attendance_ref.set(attendance_data)


### Listing events from calendar

In [34]:
googleCalendarId = "dcromp88@googlemail.com"


SCOPES = ['https://www.googleapis.com/auth/calendar']
SERVICE_ACCOUNT_FILE = '../firebase-adminsdk.json'

### Add events to Firebase [events] collection

In [36]:
from dateutil.rrule import rrulestr
from dateutil.parser import parse
import datetime
import os
import firebase_admin
from firebase_admin import credentials, firestore

os.environ["GCLOUD_PROJECT"] = "wallet-login-45c1c"

from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google.oauth2 import service_account


def get_next_occurrence(rrule_str, dtstart):
    rrule = rrulestr(rrule_str.replace('RRULE:', ''), dtstart=dtstart)
    next_occurrence = rrule.after(datetime.datetime.now(datetime.timezone.utc))
    return next_occurrence.isoformat()

def import_events():
    creds = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)

    service = build('calendar', 'v3', credentials=creds)

    now = datetime.datetime.utcnow().isoformat() + 'Z'
    print('Getting the upcoming 10 events')
    events_result = service.events().list(calendarId=googleCalendarId, timeMin=now,
                                          maxResults=10, singleEvents=False).execute()
    events = events_result.get('items', [])

    if not firebase_admin._apps:
        firebase_cred = credentials.Certificate("../firebase-adminsdk.json")
        default_app = firebase_admin.initialize_app(firebase_cred)

    db = firestore.client()

    for event in events:
        is_recurring = 'recurrence' in event
        event_id = event['id']
        summary = event.get('summary', '')
        location = event.get('location', '')
        recurrence_rules = event.get('recurrence', []) if is_recurring else []
        start = event['start'].get('dateTime', event['start'].get('date'))

        next_occurrence = None
        if is_recurring and recurrence_rules:
            next_occurrence = get_next_occurrence(recurrence_rules[0], parse(start))
        else:
            next_occurrence = start

        event_info = {
            'id': event_id,
            'is_recurring': is_recurring,
            'recurrence_rules': recurrence_rules,
            'summary': summary,
            'location': location,
            'start': start,
            'next_occurrence': next_occurrence,
        }

        event_ref = db.collection('events').document(event_id)
        event_ref.set(event_info)
        print(event_info, ' added')


if __name__ == '__main__':
    import_events()


Getting the upcoming 10 events
{'id': '4d06bu8godice3gdt7qs8k3cg4', 'is_recurring': True, 'recurrence_rules': ['RRULE:FREQ=MONTHLY;BYDAY=3SA'], 'summary': 'Competition ', 'location': '', 'start': '2023-05-20T13:00:00+02:00', 'next_occurrence': '2023-07-15T13:00:00+02:00'}  added
{'id': '_8d136ea184o42ba36l1jeb9k8kqjaba264qjgba36t2jgdhp8gq44ha66s', 'is_recurring': False, 'recurrence_rules': [], 'summary': 'Fathers day', 'location': '', 'start': '2023-11-12', 'next_occurrence': '2023-11-12'}  added
{'id': '2ai848el5g91a93m61cvc9uesm', 'is_recurring': True, 'recurrence_rules': ['RRULE:FREQ=WEEKLY;WKST=SU;BYDAY=FR,TU'], 'summary': 'Training', 'location': '', 'start': '2023-05-16T19:00:00+02:00', 'next_occurrence': '2023-07-04T19:00:00+02:00'}  added
{'id': '6cm8ofu930v8ijkkomtf6ktjdf', 'is_recurring': True, 'recurrence_rules': ['RRULE:FREQ=WEEKLY;BYDAY=FR'], 'summary': 'Adult Training', 'location': '', 'start': '2023-05-19T20:00:00+02:00', 'next_occurrence': '2023-07-07T20:00:00+02:00'}  a

In [5]:
firestore_client = firestore.client()
# Create a collection named 'groups'
groups_ref = firestore_client.collection('groups')
# Create a document with ID 'group1' and some data
group1_ref = groups_ref.document('group1')
group1_ref.set({
    'name': 'Group 1',
    'members': ['Alice', 'Bob', 'Charlie']
})

# Create another document with ID 'group2' and some data
group2_ref = groups_ref.document('group2')
group2_ref.set({
    'name': 'Group 2',
    'members': ['David', 'Eve', 'Frank']
})

### Create test events manually
Testing the strucutre

In [36]:
# Create a collection named 'events'
events_ref = firestore_client.collection('events')

# Create a document with ID 'event1' and a reference to group1
event1_ref = events_ref.document('event1')
event1_ref.set({
    'name': 'Event 1',
    'date': '2023-06-30',
    'group': group1_ref # Reference to group1 document
})

# Create another document with ID 'event2' and a reference to group2
event2_ref = events_ref.document('event2')
event2_ref.set({
    'name': 'Event 2',
    'date': '2023-07-01',
    'group': group2_ref # Reference to group2 document
})

update_time {
  seconds: 1687533427
  nanos: 938552000
}

### OLD VERSION OF EVENTS

In [ ]:
# Create a sub-collection for events under the club document
events = club_ref.collection("events")

# Create a document for an event with some data fields
event_id = "456"
event_data = {
    "title": "Fencing Tournament",
    "date": "2023-06-30",
    "location": "Gladius Arena"
}
event_ref = events.document(event_id)
event_ref.set(event_data)

# Create a sub-collection for occurrences under the event document
occurrences = event_ref.collection("occurrences")

# Create a document for an occurrence with some data fields
occurrence_id = "789"
occurrence_data = {
    "start_time": "14:00",
    "end_time": "16:00",
    "participants": ["Alice", "Bob"]
}
occurrence_ref = occurrences.document(occurrence_id)
occurrence_ref.set(occurrence_data)

# Create a sub-collection for attendance under the occurrence document
attendance = occurrence_ref.collection("attendance")

# Create a document for an attendance with some data fields
attendance_id = "1011"
attendance_data = {
    "name": "Alice",
    "score": 3
}
attendance_ref = attendance.document(attendance_id)
attendance_ref.set(attendance_data)

update_time {
  seconds: 1687370617
  nanos: 317941000
}